# Testing a Pre-Trained Network

Now that we know how to train an LSTM network, let's see try seeing the results on our own input text!

As we showed at the end of the Oriole notebook, we can load in a pretrained LSTM network using Tensorflow's Saver object. Before creating that object, we first have to create our Tensorflow graph. 

First, we declare some of our hyperparamters. 

In [1]:
numDimensions = 300
maxSeqLength = 250
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

Then, we'll load in our data structures.

In [2]:
import numpy as np
wordsList = np.load('wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')

Next, we'll create our graph. This is the same code as we saw in the main Oriole notebook. 

In [3]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

Now, we load in the network. 

In [9]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models\pretrained_lstm-90000.ckpt


C:\Users\thoan\Miniconda3\lib\site-packages\tensorflow\python\client\session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Before we input our own text, let's first define a couple of functions. The first is a function to make sure the sentence is in the proper format, and the second is a function that obtains the word vectors for each of the words in a given sentence. 

In [10]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

Now, we can create our input text. Try modifying the value for inputText, and see how the outputs change!

In [11]:
inputText = "That movie was terrible."
inputMatrix = getSentenceMatrix(inputText)

In [14]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment

if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Negative Sentiment


In [15]:
secondInputText = "That movie was the best one I have ever seen."
secondInputMatrix = getSentenceMatrix(secondInputText)

In [17]:
predictedSentiment = sess.run(prediction, {input_data: secondInputMatrix})[0]
if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Positive Sentiment


In [18]:
import pandas as pd

In [36]:
df = pd.read_csv("data_new_2.csv")
df

,Unnamed: 0,ID,Hash,Time,Content,Latitude,Longitude,Brand,Continent
0,0,1.910000e+18,BpwYvMDnR7A,11/4/2018 11:03,"La notte ha portato pugni,calci e testate\nMa ...",0.000000,0.100000,Nutella,EU & Africa
1,1,1.910000e+18,BpwYs0mHNtW,11/4/2018 11:02,Petit déjeuner à l'américain󾓦\n\n#pancakes #co...,0.000000,0.000000,Nutella,EU & Africa
2,2,1.910000e+18,BpwYlOJHFDL,11/4/2018 11:01,Sabato. ✨🍫\n#piccolegioie\n\n#saturdaymood #sa...,44.034378,10.137616,Nutella,EU & Africa
3,3,1.910000e+18,BpwYjKvAd_W,11/4/2018 11:01,#nutella#lı kahve denediniz mi 😘mutlu pazarlar 💕,38.013333,32.698889,Nutella,EU & Africa
4,4,1.910000e+18,BpwYiB-g7Y6,11/4/2018 11:01,"-""more mi prepari un toast con la #nutella ?""-...",0.000000,0.000000,Nutella,EU & Africa
5,5,1.910000e+18,BpwYeLHlN5p,11/4/2018 11:00,Kanel- och kardemummavåfflor serveras med #nut...,59.875237,17.648617,Nutella,EU & Africa
6,6,1.910000e+18,BpwYeAlDMuo,11/4/2018 11:00,Les petites crêpes du dimanche matin 🥞 🤗\n✖️\n...,0.000000,0.000000,Nutella,EU & Africa
7,7,1.910000e+18,BpwYcFpnp_5,11/4/2018 11:00,Black Friday Novembro\nNo delivery Ifood \n#if...,-23.561210,-46.691960,Nutella,America
8,8,1.910000e+18,BpwYXwYFI2v,11/4/2018 10:59,#anniversary #cupcakes as ordered 😋❤️,0.000000,0.000000,Nutella,EU & Africa
9,9,1.910000e+18,BpwYSjFl_Kk,11/4/2018 10:59,Bonjour 😍 un petit déjeuner gourmand 🤩\nBon di...,48.856700,2.350800,Nutella,EU & Africa


In [37]:
content = list(df.Content)
content

["La notte ha portato pugni,calci e testate\nMa non c'è risveglio migliore\nDomeniche📷❤️😇😈\n#sundaymorning #breakfast #colazionideicampioni #nutella #buondì #lapartita #mattia #angioletto #diavoletto #instabreakfast #piç #felicita",
 "Petit déjeuner à l'américain\U000fe4e6\n\n#pancakes #coffee #beignets #orange #siroperable #nutella #breakfast #sunday #withlove #picoftheday",
 'Sabato. ✨🍫\n#piccolegioie\n\n#saturdaymood #saturdaynight #food #foodlover #fooporn #streetfood #sempreamangiare #waffle #nutella #magraMAI #moodoftheday #vsco #vscocam #picoftheday',
 '#nutella#lı kahve denediniz mi 😘mutlu pazarlar 💕',
 '-"more mi prepari un toast con la #nutella ?"-\n•\n•\n•\n•\nApri ed esce qst! •\n•\n•\n•\n#iloveyou #iloveyoumylove #instalove\n#instanoi #andreaegiulia #mansardapassariellotoscano',
 'Kanel- och kardemummavåfflor serveras med #nutella 😋 #ätabörman #hemlagat',
 'Les petites crêpes du dimanche matin 🥞 🤗\n✖️\n#dimanche #dimanchematin #bondimanche #dimancheaucalme #dimanchealamais

In [38]:
result = []

In [39]:
for c in content:
    #inputText = "That movie was terrible."
    inputMatrix = getSentenceMatrix(c)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment

    if (predictedSentiment[0] > predictedSentiment[1]):
        print("Positive Sentiment")
        result.append("Positive")
    else:
        print("Negative Sentiment")
        result.append("Negative")

Positive Sentiment
Positive Sentiment
Negative Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Negative Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Negative Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Negative Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Negative Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Positive Sentiment
Positive Sen

In [40]:
ret_df = pd.DataFrame(content, result)

In [41]:
print(ret_df)

                                                          0
Positive  La notte ha portato pugni,calci e testate\nMa ...
Positive  Petit déjeuner à l'américain󾓦\n\n#pancakes #co...
Negative  Sabato. ✨🍫\n#piccolegioie\n\n#saturdaymood #sa...
Negative   #nutella#lı kahve denediniz mi 😘mutlu pazarlar 💕
Positive  -"more mi prepari un toast con la #nutella ?"-...
Negative  Kanel- och kardemummavåfflor serveras med #nut...
Positive  Les petites crêpes du dimanche matin 🥞 🤗\n✖️\n...
Negative  Black Friday Novembro\nNo delivery Ifood \n#if...
Positive              #anniversary #cupcakes as ordered 😋❤️
Positive  Bonjour 😍 un petit déjeuner gourmand 🤩\nBon di...
Positive  Mad over Shakes is one of the best palce for t...
Negative  Happy b-day to me~ 😂❤🔥 #colazione #breakfast #...
Positive  ...E quando il tuo bimbo chiede cioccolato den...
Negative  If i must work on a Sunday 🤣😍☕️🥜🌰 #catchingupo...
Positive  Cold mornings are made for porridge and #nutel...
Positive  We love pancakes 🥞 almost ever

In [42]:
df['result'] = result

In [43]:
df

,Unnamed: 0,ID,Hash,Time,Content,Latitude,Longitude,Brand,Continent,result
0,0,1.910000e+18,BpwYvMDnR7A,11/4/2018 11:03,"La notte ha portato pugni,calci e testate\nMa ...",0.000000,0.100000,Nutella,EU & Africa,Positive
1,1,1.910000e+18,BpwYs0mHNtW,11/4/2018 11:02,Petit déjeuner à l'américain󾓦\n\n#pancakes #co...,0.000000,0.000000,Nutella,EU & Africa,Positive
2,2,1.910000e+18,BpwYlOJHFDL,11/4/2018 11:01,Sabato. ✨🍫\n#piccolegioie\n\n#saturdaymood #sa...,44.034378,10.137616,Nutella,EU & Africa,Negative
3,3,1.910000e+18,BpwYjKvAd_W,11/4/2018 11:01,#nutella#lı kahve denediniz mi 😘mutlu pazarlar 💕,38.013333,32.698889,Nutella,EU & Africa,Negative
4,4,1.910000e+18,BpwYiB-g7Y6,11/4/2018 11:01,"-""more mi prepari un toast con la #nutella ?""-...",0.000000,0.000000,Nutella,EU & Africa,Positive
5,5,1.910000e+18,BpwYeLHlN5p,11/4/2018 11:00,Kanel- och kardemummavåfflor serveras med #nut...,59.875237,17.648617,Nutella,EU & Africa,Negative
6,6,1.910000e+18,BpwYeAlDMuo,11/4/2018 11:00,Les petites crêpes du dimanche matin 🥞 🤗\n✖️\n...,0.000000,0.000000,Nutella,EU & Africa,Positive
7,7,1.910000e+18,BpwYcFpnp_5,11/4/2018 11:00,Black Friday Novembro\nNo delivery Ifood \n#if...,-23.561210,-46.691960,Nutella,America,Negative
8,8,1.910000e+18,BpwYXwYFI2v,11/4/2018 10:59,#anniversary #cupcakes as ordered 😋❤️,0.000000,0.000000,Nutella,EU & Africa,Positive
9,9,1.910000e+18,BpwYSjFl_Kk,11/4/2018 10:59,Bonjour 😍 un petit déjeuner gourmand 🤩\nBon di...,48.856700,2.350800,Nutella,EU & Africa,Positive


In [44]:
df.to_csv('data_new.csv', sep=',')